In [ ]:
%pip install promptsource==0.2.3

# Prompt templates

https://github.com/bigscience-workshop/promptsource/blob/main/promptsource/templates/amazon_us_reviews/Wireless_v1_00/templates.yaml

In [3]:
from promptsource.templates import DatasetTemplates

prompt_templates = DatasetTemplates('amazon_us_reviews/Wireless_v1_00')

In [4]:
for template in prompt_templates.templates.values():
    print(template.get_name())

Generate review headline based on review body
Generate review based on rating and category
Given the review headline return a categorical rating
Generate review headline based on rating
Given the review body return a categorical rating


In [5]:
prompt = prompt_templates["Given the review body return a categorical rating"]
print(prompt.answer_choices)

1 ||| 2 ||| 3 ||| 4 ||| 5


In [6]:
print(prompt.__dict__)

{'answer_choices': '1 ||| 2 ||| 3 ||| 4 ||| 5', 'id': 'e6a1bbde-715d-4dad-9178-e2bcfaf5c646', 'jinja': "Given the following review:\n{{review_body}}\npredict the associated rating from the following choices (1 being lowest and 5 being highest)\n- {{ answer_choices | join('\\n- ') }} \n|||\n{{answer_choices[star_rating-1]}}", 'metadata': <promptsource.templates.Template.Metadata object at 0x7ff1b057e990>, 'name': 'Given the review body return a categorical rating', 'reference': 'Given the review body, return a categorical rating. '}


In [7]:
from datasets import load_dataset
dataset = load_dataset("amazon_us_reviews", "Digital_Software_v1_00", split="train")

Found cached dataset amazon_us_reviews (/root/.cache/huggingface/datasets/amazon_us_reviews/Digital_Software_v1_00/0.1.0/17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563)


In [10]:
example = dataset[1]
print(example)

{'marketplace': 'US', 'customer_id': '10956619', 'review_id': 'R1W5OMFK1Q3I3O', 'product_id': 'B00HRJMOM4', 'product_parent': '162269768', 'product_title': 'ResumeMaker Professional Deluxe 18', 'product_category': 'Digital_Software', 'star_rating': 3, 'helpful_votes': 0, 'total_votes': 0, 'vine': 0, 'verified_purchase': 1, 'review_headline': 'Three Stars', 'review_body': 'Needs a little more work.....', 'review_date': '2015-08-31'}


In [14]:
result = prompt.apply(example)

In [15]:
print("INPUT: ", result[0])

INPUT:  Given the following review:
Needs a little more work.....
predict the associated rating from the following choices (1 being lowest and 5 being highest)
- 1
- 2
- 3
- 4
- 5


In [18]:
print("TARGET: ", result[1])

TARGET:  3


In [21]:
dataset.select([10, 20]).map(lambda row : {'prompt': '\n\nPROMPT: ' + prompt.apply(row)[0] + '\nTARGET: ' + prompt.apply(row)[1]})

  0%|          | 0/2 [00:00<?, ?ex/s]

Dataset({
    features: ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date', 'prompt'],
    num_rows: 2
})

In [28]:
prompt0 = prompt.apply(dataset[0])
prompt1 = prompt.apply(dataset[1])
prompt2 = prompt.apply(dataset[2])
prompt3 = prompt.apply(dataset[3])

In [33]:
few_shot_prompt = 'PROMPT: ' + prompt0[0] + '\nSTAR_RATING: ' + prompt0[1] + '\n\nPROMPT: ' + prompt1[0] + '\nSTAR_RATING: ' + prompt1[1] + '\n\nPROMPT: ' + prompt2[0] + '\nSTAR_RATING: ' + prompt2[1] + '\n\PROMPT: ' + prompt3[0] + '\nSTAR_RATING: '
print(few_shot_prompt)

PROMPT: Given the following review:
So far so good
predict the associated rating from the following choices (1 being lowest and 5 being highest)
- 1
- 2
- 3
- 4
- 5
STAR_RATING: 4

PROMPT: Given the following review:
Needs a little more work.....
predict the associated rating from the following choices (1 being lowest and 5 being highest)
- 1
- 2
- 3
- 4
- 5
STAR_RATING: 3

PROMPT: Given the following review:
Please cancel.
predict the associated rating from the following choices (1 being lowest and 5 being highest)
- 1
- 2
- 3
- 4
- 5
STAR_RATING: 1
\PROMPT: Given the following review:
Works as Expected!
predict the associated rating from the following choices (1 being lowest and 5 being highest)
- 1
- 2
- 3
- 4
- 5
STAR_RATING: 


In [ ]:
result_length = len(few_shot_prompt + 2)
inputs = tokenizer(few_shot_prompt, return_tensors='pt')

print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_length=result_length, 
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))